TODO

make employee count based on distribution: https://stackoverflow.com/questions/4265988/generate-random-numbers-with-a-given-numerical-distribution
i.e. realistic shift-employee ratio

In [2]:
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
import glob

In [51]:
def matrify(pool, schedule):
    schedule = pd.get_dummies(schedule,drop_first=True)
    #shift_features = schedule.shape[1]
    for i in pd.get_dummies(pool).columns.to_list():
        schedule[i] = 0

    return schedule.to_numpy()

In [57]:
# is problem solvable 

pi = pd.read_csv('problem_index.csv',dtype=str) 
pi.head()

glob_list = []
for i in range(len(pi)):
    s = pi['Schedule'].iloc[i]
    p = pi['Pool'].iloc[i]
    glob_list.append((s,p))

s = glob_list[0][0]
p = glob_list[0][1]

pool, schedule = pd.read_csv(f'pools/pool_{p}.csv',dtype={'employee_id':'str'}), \
                pd.read_csv(f'schedules/schedule_{s}.csv',dtype={'shift_id':'str'})

#schedule['shift_day_of_week'] = schedule['shift_day_of_week'].replace(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'],[1, 2, 3, 4, 5])
#schedule['shift_type'] = schedule['shift_type'].replace(['Morning', 'Evening'],[1, 2])

matrify(pool, schedule)

# simulatenous shifts + the next consecutive shift

# less than workers

# return minimum workers for each shift

array([[0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0]])

In [58]:
schedule

,shift_id,shift_day_of_week,shift_type
0,0,Monday,Morning
1,1,Monday,Evening
2,2,Tuesday,Morning
3,3,Tuesday,Evening


In [48]:
scheds = sorted(glob.glob("schedules/*.csv"))
id = int(scheds[-1].split('/')[1].split('_')[1].split('.')[0]) # .split('/')[2]
id
scheds[-1]

'schedules/schedule_0058.csv'

In [30]:
def scheduleValidation():
        #check for duplicates in schedules

        scheds = glob.glob("schedules/*.csv") # "scheduling_problems/schedules/*.csv"

        schedDict = {}
        for sched in scheds:
                id = sched.split('/')[1].split('_')[1].split('.')[0]  # .split('/')[2]
                s = pd.read_csv(sched,dtype={'shift_id':'str'})
                schedDict[id] = s

        dupes=[]
        for key, value in schedDict.items(): 
                for k, v in schedDict.items(): 
                        if key != k:
                                if value.equals(v):
                                        dupes.append((key,k))

        return dupes


In [42]:
scheduleValidation()

[]

In [39]:
def randomSchedule(n=1, min_shifts=2,max_shifts=16, max_shifts_per_day=4):
    for i in range(n):
        # shift_id
        n = random.randint(min_shifts, max_shifts)
        shift_id = list(range(0, n))

        shifts = len(shift_id)


        # shift_day_of_week
        shift_day_of_week = []
        days = ['Monday','Tuesday','Wednesday','Thursday','Friday']

        for i in range(len(shift_id)):
            choice = random.choice(days)
            if shift_day_of_week.count(choice) < max_shifts_per_day + 1:
                shift_day_of_week.append(choice)

        shift_day_of_week = sorted(shift_day_of_week, key=days.index)

        # shift_type
        shift_type = []
        shift_types = ['Morning','Evening']

        dayset = set(shift_day_of_week)
        sortedDayset = sorted(list(dayset), key=days.index)

        for i in sortedDayset:
            temp = []
            for j in range(shift_day_of_week.count(i)):
                temp.append(random.choice(shift_types))
            temp = sorted(temp, key=shift_types.index)
            for k in temp:
                shift_type.append(k)

        scheduleDic = {'shift_id':shift_id,'shift_day_of_week':shift_day_of_week,'shift_type':shift_type}

        schedule = pd.DataFrame(scheduleDic)

        id = int(scheds[-1].split('/')[2].split('_')[1].split('.')[0]) # .split('/')[1]

        schedule.to_csv(f'schedules/schedule_00{id}.csv',index=False)

        print(f"schedule_00{id} saved.")

    #return schedule
        

In [41]:
randomSchedule()

,shift_id,shift_day_of_week,shift_type
0,0,Monday,Evening
1,1,Tuesday,Morning
2,2,Wednesday,Morning
3,3,Wednesday,Evening


## BUILDING TEST SET 

In [60]:
def empGen(num_shifts,num_emps=False):
    employee_id = []
    e = random.randint(2, num_shifts)
    if num_emps:
        for i in range(num_emps):
            employee_id.append(''.join(np.random.randint(9,size=(6)).astype('str')))
    else:
        for i in range(e):
            employee_id.append(''.join(np.random.randint(9,size=(6)).astype('str')))
    
    return employee_id


def randomProblem(min_shifts=2,max_shifts=15, max_shifts_per_day=4,num_emps=False):
    
    # shift_id
    n = random.randint(min_shifts, max_shifts)
    shift_id = list(range(0, n))
    
    # shift_day_of_week
    shift_day_of_week = []
    days = ['Monday','Tuesday','Wednesday','Thursday','Friday']

    for i in range(len(shift_id)):
        choice = random.choice(days)
        if shift_day_of_week.count(choice) < max_shifts_per_day + 1:
            shift_day_of_week.append(choice)

    shift_day_of_week = sorted(shift_day_of_week, key=days.index)
    
    # shift_type
    shift_type = []
    shift_types = ['Morning','Evening']

    dayset = set(shift_day_of_week)
    sortedDayset = sorted(list(dayset), key=days.index)

    for i in sortedDayset:
        temp = []
        for j in range(shift_day_of_week.count(i)):
            temp.append(random.choice(shift_types))
        temp = sorted(temp, key=shift_types.index)
        for k in temp:
            shift_type.append(k)
            
    # employee_id
    while True:
        employee_id = empGen(len(shift_id),num_emps)
        if len(shift_id) / len(employee_id) <= 5:
            break
        
    scheduleDic = {'shift_id':shift_id,'shift_day_of_week':shift_day_of_week,'shift_type':shift_type}
    
    schedule = pd.DataFrame(scheduleDic)
    pool = pd.DataFrame({'employee_id':employee_id})
    
    return schedule, pool

In [144]:
def buildTestSet(n,min_shifts,max_shifts,num_emps):
    for i in range(n):
        try:
            schedule, pool = randomProblem(min_shifts=min_shifts,max_shifts=max_shifts,num_emps=num_emps)

            #test for ratio?

            tstst = sorted(glob.glob("test_set/*.csv"))

            if tstst:
                id = int(tstst[-1].split('/')[1].split('_')[1].split('.')[0]) + 1
        
            else:
                id = 1

            schedule, pool = schedule.to_csv(f'test_set/schedule_{str(id).zfill(2)}.csv',index=False), \
                            pool.to_csv(f'test_set/pool_{str(id).zfill(2)}.csv',index=False) 
        
        except:
            pass


In [ ]:
buildTestSet(n=50,min_shifts=3,max_shifts=14,num_emps=6)

In [137]:
def loadTestProblem(num_shifts=False):
    tstst = sorted(glob.glob("test_set/*.csv"))

    try:
        if num_shifts:
            shfts = 0
            while not shfts == num_shifts:
                n = random.randint(1, (len(tstst) / 2))
                p = tstst[n-1]
                s = tstst[n+(int(len(tstst) / 2)-1)]

                schedule = pd.read_csv(f'{tstst[n+(int(len(tstst) / 2)-1)]}',dtype={'shift_id':'str'})
                if len(schedule) == num_shifts:
                    shfts = num_shifts

        else:
            n = random.randint(1, (len(tstst) / 2))
            p = tstst[n-1]
            s = tstst[n+(int(len(tstst) / 2)-1)]

        return (p,s)
    except:
        print(f"No problem with {num_shifts} shifts.")

In [150]:
loadTestProblem(num_shifts=5)

('test_set/pool_17.csv', 'test_set/schedule_17.csv')

In [106]:
tstst = sorted(glob.glob("test_set/*.csv"))
n = random.randint(1, (len(tstst) / 2))

pool, schedule = pd.read_csv(f'{tstst[n-1]}',dtype={'employee_id':'str'}), \
                pd.read_csv(f'{tstst[n+(int(len(tstst) / 2)-1)]}',dtype={'shift_id':'str'})

pool,schedule

(  employee_id
 0      352862
 1      630418
 2      187636
 3      038445
 4      065066
 5      353246,
    shift_id shift_day_of_week shift_type
 0         0            Monday    Evening
 1         1            Monday    Evening
 2         2            Monday    Evening
 3         3           Tuesday    Morning
 4         4           Tuesday    Morning
 5         5           Tuesday    Evening
 6         6         Wednesday    Evening
 7         7         Wednesday    Evening
 8         8         Wednesday    Evening
 9         9          Thursday    Morning
 10       10          Thursday    Evening
 11       11          Thursday    Evening
 12       12          Thursday    Evening
 13       13            Friday    Evening
 14       14            Friday    Evening)

In [108]:
tstst[n+(int(len(tstst) / 2)-1)]
#tstst[n-1]



'test_set/schedule_01.csv'

In [80]:
# need an overview of the test set

#tpi = pd.DataFrame(columns=['Schedule', 'Pool'])

'12'

In [ ]:
#######

In [6]:
scheds[0].split('/')[1].split('_')[1].split('.')[0]

'0012'

In [28]:
for i in range(40):
    sid = len(glob.glob("schedules/*.csv")) + 1
    pid = len(glob.glob("pools/*.csv")) + 1
    schedule, pool = randomProblem(min_shifts=8,max_shifts=8,num_emps=4)
    schedule.to_csv(f"schedules/schedule_00{sid}.csv",index=False)
    pool.to_csv(f"pools/pool_00{str(pid).zfill(2)}.csv",index=False)
